# ASSIGNMENT 2
Assigment is in the intersection of Named Entity Recognition and Dependency Parsing.

0. Evaluate spaCy NER on CoNLL 2003 data (provided)
    - report token-level performance (per class and total)
        - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
    - report CoNLL chunk-level performance (per class and total);
        - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total  

1. Grouping of Entities.
Write a function to group recognized named entities using `noun_chunks` method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks). Analyze the groups in terms of most frequent combinations (i.e. NER types that go together). 

2. One of the possible post-processing steps is to fix segmentation errors.
Write a function that extends the entity span to cover the full noun-compounds. Make use of `compound` dependency relation.

# 0) Start

- First we need to import everything we will ned to use. 

- I imported every function from the "conll.py" file, because I decided to use the "read_corpus_conll" function to read the conll2003 file. 

- I find alse the Class WhitespaceTokenizer in the SpaCy ducumentation to have a more effective tokenization from string 

- I will use classification_report of sklearn.metrics to evaluete the accuracy

In [32]:
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))
import spacy
import pandas as pd
from conll import *
from spacy.tokens import Doc
from spacy.vocab import Vocab
from sklearn.metrics import classification_report


# taken from https://spacy.io/usage/linguistic-features#tokenization
class WhitespaceTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(" ")
        return Doc(self.vocab, words=words)
    

In [35]:
nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)


# Preparation

In [33]:
def get_sents(path):
    
    sents=read_corpus_conll(path)
    
    sents.remove(sents[0]) #removing '-DOCSTART- -X- -X- O' line
    #print(sents)
    return sents
    
path1='../src/conll2003/test.txt'
path2='../src/conll2003/train.txt'

sents=get_sents(path1)
print(sents[0:2])

[[('SOCCER NN B-NP O',), ('- : O O',), ('JAPAN NNP B-NP B-LOC',), ('GET VB B-VP O',), ('LUCKY NNP B-NP O',), ('WIN NNP I-NP O',), (', , O O',), ('CHINA NNP B-NP B-PER',), ('IN IN B-PP O',), ('SURPRISE DT B-NP O',), ('DEFEAT NN I-NP O',), ('. . O O',)], [('Nadim NNP B-NP B-PER',), ('Ladki NNP I-NP I-PER',)]]


In [34]:
def get_sent_string(corpus):
    temp=[]
    temp2=[]
    t_sent=[] #['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT']..
    f_sent=[] #'SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT'
    s_iobsent=[] #['SOCCER', 'NN', 'B-NP', 'O'], ['-', ':', 'O', 'O'] ...
    for s in sents: 
        for w in s:
            if (w[0].split()[0]!='.'):
                temp.append(w[0].split()[0])
                temp2.append(w[0].split())

            else:
                t_sent.append(temp)
                s_iobsent.append(temp2)
                temp=[]
                temp2=[]

    for s in t_sent:
        f_sent.append(' '. join(s))

    return f_sent,s_iobsent

s, iob= get_sent_string(sents)

print (s[0])
print (iob[0])

SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT
[['SOCCER', 'NN', 'B-NP', 'O'], ['-', ':', 'O', 'O'], ['JAPAN', 'NNP', 'B-NP', 'B-LOC'], ['GET', 'VB', 'B-VP', 'O'], ['LUCKY', 'NNP', 'B-NP', 'O'], ['WIN', 'NNP', 'I-NP', 'O'], [',', ',', 'O', 'O'], ['CHINA', 'NNP', 'B-NP', 'B-PER'], ['IN', 'IN', 'B-PP', 'O'], ['SURPRISE', 'DT', 'B-NP', 'O'], ['DEFEAT', 'NN', 'I-NP', 'O']]


# Create doc for all the sentencies


In [73]:
sentences=[]

for i in s: 
    doc=nlp(i)
    sentences.append(doc)

# From SpaCy format to ConLL 

1. There are four types of phrases in CoNLL format: person names (PER), organizations (ORG), locations (LOC) and miscellaneous names (MISC)


2. Spacy (Named Entity Recognition, OntoNotes 5, https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf)
    
    - "PERSON": "People, including fictional",
    - "NORP": "Nationalities or religious or political groups",
    - "FACILITY": "Buildings, airports, highways, bridges, etc.",
    - "FAC": "Buildings, airports, highways, bridges, etc.",
    - "ORG": "Companies, agencies, institutions, etc.",
    - "GPE": "Countries, cities, states",
    - "LOC": "Non-GPE locations, mountain ranges, bodies of water",
    - "PRODUCT": "Objects, vehicles, foods, etc. (not services)",
    - "EVENT": "Named hurricanes, battles, wars, sports events, etc.",
    - "WORK_OF_ART": "Titles of books, songs, etc.",
    - "LAW": "Named documents made into laws.",
    - "LANGUAGE": "Any named language",
    - "DATE": "Absolute or relative dates or periods",
    - "TIME": "Times smaller than a day",
    - "PERCENT": 'Percentage, including "%"',
    - "MONEY": "Monetary values, including unit",
    - "QUANTITY": "Measurements, as of weight or distance",
    - "ORDINAL": '"first", "second", etc.',
    - "CARDINAL": "Numerals that do not fall under another type",



3. I defined the function creating tha hypotesis that get in input a list of strings anche return a list of tokenised and elaborate tuples 
    - input: ['SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT' , '...']
    - output: [[(SOCCER, 'O'), (-, 'O'), (JAPAN, 'O'), (GET, 'O'), (LUCKY, 'O'), (WIN, 'O'), (,, 'O'), (CHINA, 'B-LOC'), (IN, 'O'), (SURPRISE, 'O'), (DEFEAT, 'O')], [...]]

In [74]:
dictionary = {
    'PERSON': 'PER',
    'NORP': 'MISC',
    'FAC': 'ORG',
    'ORG': 'ORG',
    'GPE': 'LOC',
    'LOC': 'LOC',
    'PRODUCT': 'MISC',
    'EVENT': 'MISC',
    'WORK_OF_ART': 'PER' ,
    'LAW': 'MISC',
    'LANGUAGE': 'MISC',
    'DATE': 'MISC',
    'TIME': 'MISC',
    'PERCENT': 'MISC',
    'MONEY': 'MISC',
    'QUANTITY': 'MISC',
    'ORDINAL': 'MISC',
    'CARDINAL': 'MISC',
}


def get_hyp(sents):

    
    hip=[]
    temp=[]

    for s in sents:
        for t in s:
            if t.ent_type_ in dictionary.keys():
                x=t.ent_iob_+ '-'+dictionary[t.ent_type_]
                temp.append((t, x))
            else: 
                temp.append((t, t.ent_iob_))
        hip.append(temp)
        temp=[]

    return hip

hyp=get_hyp(sentences)
print (hyp[0:2])

[[(SOCCER, 'O'), (-, 'O'), (JAPAN, 'O'), (GET, 'O'), (LUCKY, 'O'), (WIN, 'O'), (,, 'O'), (CHINA, 'B-LOC'), (IN, 'O'), (SURPRISE, 'O'), (DEFEAT, 'O')], [(Nadim, 'B-ORG'), (Ladki, 'I-ORG'), (AL-AIN, 'O'), (,, 'O'), (United, 'B-LOC'), (Arab, 'I-LOC'), (Emirates, 'I-LOC'), (1996-12-06, 'O'), (Japan, 'B-LOC'), (began, 'O'), (the, 'O'), (defence, 'O'), (of, 'O'), (their, 'B-MISC'), (Asian, 'I-MISC'), (Cup, 'I-MISC'), (title, 'O'), (with, 'O'), (a, 'O'), (lucky, 'O'), (2-1, 'B-MISC'), (win, 'O'), (against, 'O'), (Syria, 'B-LOC'), (in, 'O'), (a, 'O'), (Group, 'B-ORG'), (C, 'I-ORG'), (championship, 'O'), (match, 'O'), (on, 'O'), (Friday, 'B-MISC')]]


# Getting 'refs'

from the function 'get_sent_string(corpus)' I returned the concatenized string but alza the plit string with all the other parameters, so is I take that variable (iob) and remove the part from each list of each sentences that I don't need for the comparison

In [75]:
print (iob[0])

[['SOCCER', 'NN', 'B-NP', 'O'], ['-', ':', 'O', 'O'], ['JAPAN', 'NNP', 'B-NP', 'B-LOC'], ['GET', 'VB', 'B-VP', 'O'], ['LUCKY', 'NNP', 'B-NP', 'O'], ['WIN', 'NNP', 'I-NP', 'O'], [',', ',', 'O', 'O'], ['CHINA', 'NNP', 'B-NP', 'B-PER'], ['IN', 'IN', 'B-PP', 'O'], ['SURPRISE', 'DT', 'B-NP', 'O'], ['DEFEAT', 'NN', 'I-NP', 'O']]


In [76]:
def get_refs(corpus):
    sent, iob= get_sent_string(corpus)
    for s in iob: 
        for t in s: 
            t.remove(t[1])
            t.remove(t[1])

    return iob

refs=get_refs(sents)
print (refs[0])

[['SOCCER', 'O'], ['-', 'O'], ['JAPAN', 'B-LOC'], ['GET', 'O'], ['LUCKY', 'O'], ['WIN', 'O'], [',', 'O'], ['CHINA', 'B-PER'], ['IN', 'O'], ['SURPRISE', 'O'], ['DEFEAT', 'O']]


# 0.1) Token-Level Accuracy

- I take the iob information in the hyp and refs, save it in two list of "output" and "predicted", I then use classification_ report to confront them 

(As for piazza question of "Steve Azzolin" I used for example B-ORG as a different class wrt I-ORG and I solso keeped the tagged O)

- From sklearn: 
    - The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
    - The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
    - The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:
        - F1 = 2 * (precision * recall) / (precision + recall)

In [77]:
def get_accuracy(hyp, refs):
    pred = []
    output=[]
    
    for s in refs:
        for t in s:
              output.append(t[1])
    
    for s in hyp:
        for t in s:
              pred.append(t[1])


    

    return output,pred

out, pred= get_accuracy (hyp, refs)
report = classification_report(out, pred, output_dict=True)
pd_tbl = pd.DataFrame(report).transpose()
#pd_tbl.round(decimals=3)
print (pd_tbl)

              precision    recall  f1-score       support
B-LOC          0.791753  0.690647  0.737752   1668.000000
B-MISC         0.106371  0.547009  0.178108    702.000000
B-ORG          0.502083  0.290187  0.367799   1661.000000
B-PER          0.773344  0.599258  0.675261   1617.000000
I-LOC          0.593750  0.591440  0.592593    257.000000
I-MISC         0.051990  0.375000  0.091319    216.000000
I-ORG          0.428858  0.512575  0.466994    835.000000
I-PER          0.802446  0.737889  0.768815   1156.000000
O              0.938765  0.861456  0.898451  36927.000000
accuracy       0.806235  0.806235  0.806235      0.806235
macro avg      0.554373  0.578385  0.530788  45039.000000
weighted avg   0.879129  0.806235  0.836746  45039.000000


# 0.2) Chunk level performance
Using the evaluate function in conll.py

In [78]:
report = evaluate (refs, hyp)
pd_tbl = pd.DataFrame().from_dict(report, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
ORG,0.441,0.255,0.323,1661
PER,0.741,0.574,0.647,1617
LOC,0.780,0.680,0.727,1668
MISC,0.104,0.536,0.174,702
total,0.393,0.507,0.443,5648


# 1) Grouping of Entities

- I create three function (I need both because as demostrated below the numeber of entities with doc.ents and noun_chuck are differente, I use noun_chuck and discard the one that are found not found in the doc.ents and then store in one arrey ) (Also for the comment in Piazza were professor Stepanov wrote "pay attention... some entities may be not within noun_chunk spans"): 
    - the first to retive the entities with the function noun_chucks, and I return them 
    - the second to extract the entities from a doc with 'doc.ents' and confront with the previius list, and then I return them
      
- the peculiar thing is thaht noun_chuck return a Span object: 
    - Token vs Span: From spaCy's documentation, a Token represents a single word, punctuation symbol, whitespace, etc. from a document, while a Span is a slice from the document. In other words, a Span is an ordered sequence of Tokens. 
    - So u can acces the entity type with "span.root.ent_type"



In [185]:
def get_ent_doc(sents):
    ent=tuple()
    for d in sents: 
        for e in d.ents: 
            ent.append(e)
    return ent

In [186]:
def get_ent_noun(sents):
    ent=[]
    for d in sents: 
        for e in d.noun_chunks: 
            for s in e.ents: 
                ent.append(s)
    return ent

In [187]:
a=get_ent_doc(sentences)
b=get_ent_noun(sentences)

print (len(a)==len(b))

False


Now the implementation of "get_ent(sents)"

In [206]:
def get_ent(sents):
    ent=[]
    temp = []
    check=get_ent_noun(sents)
    for d in sents:
        for e in d.noun_chunks:
            for s in e.ents:
                if s in check:
                    temp.append(s.root.ent_type_)
                    
                    check.remove(s)
            ent.append(temp)
            temp=[]
    
    
    return ent

In [230]:
def get_freq(ent):
    freq={}
    for x in ent:
        if tuple(x)!=():
            if(tuple(x) not in freq):
                freq[tuple(x)] = 1
            else:
                freq[tuple(x)] += 1
    return freq

In [233]:
ent=get_ent(sentences)
f= get_freq(ent)
sorted_dict = dict( sorted(f.items(),
                           key=lambda item: item[1],
                           reverse=True))

for key, value in sorted_dict.items():
    print(key, ' : ', value)

('GPE',)  :  1121
('PERSON',)  :  894
('ORG',)  :  651
('CARDINAL',)  :  510
('DATE',)  :  472
('NORP',)  :  273
('CARDINAL', 'PERSON')  :  102
('ORDINAL',)  :  87
('PERCENT',)  :  50
('EVENT',)  :  49
('MONEY',)  :  48
('TIME',)  :  42
('LOC',)  :  40
('NORP', 'PERSON')  :  38
('QUANTITY',)  :  36
('GPE', 'PERSON')  :  35
('CARDINAL', 'ORG')  :  35
('CARDINAL', 'GPE')  :  26
('CARDINAL', 'NORP')  :  22
('CARDINAL', 'CARDINAL')  :  20
('ORG', 'PERSON')  :  20
('GPE', 'GPE')  :  18
('FAC',)  :  18
('PRODUCT',)  :  17
('GPE', 'ORG')  :  16
('GPE', 'CARDINAL')  :  12
('WORK_OF_ART',)  :  10
('DATE', 'EVENT')  :  9
('PERSON', 'PERSON')  :  9
('ORG', 'ORG')  :  9
('DATE', 'TIME')  :  7
('PERSON', 'GPE')  :  7
('GPE', 'PRODUCT')  :  7
('GPE', 'DATE')  :  6
('NORP', 'ORDINAL')  :  6
('GPE', 'ORDINAL')  :  5
('CARDINAL', 'GPE', 'GPE')  :  5
('ORG', 'GPE')  :  5
('ORG', 'DATE')  :  5
('CARDINAL', 'CARDINAL', 'CARDINAL')  :  5
('NORP', 'ORG')  :  5
('DATE', 'ORG')  :  4
('DATE', 'PERSON')  :  4


# 2) Post-processing steps